In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
import numpy as np
from sklearn import metrics

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import load_model

In [ ]:
import json
import os

from pprint import pprint

In [ ]:
from data.coco.coco import CocoGenerator

In [ ]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel

# datasets

In [ ]:
# train
train_dataset = CocoGenerator('train', '/share/DEEPLEARNING/datasets/mscoco')

In [ ]:
generator_train = train_dataset.flow(batch_size=2)
X_train, Y_train = next(generator_train)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
# validation
val_dataset = CocoGenerator('val', '/share/DEEPLEARNING/datasets/mscoco')

In [ ]:
print(len(val_dataset.image_ids_in_subset))

In [ ]:
batch_size = len(val_dataset.image_ids_in_subset)
generator_val = val_dataset.flow(batch_size=batch_size)
X_test, Y_test = next(generator_val)
print(X_test.shape, Y_test.shape)

# model

In [ ]:
# dropping the Dense(1000)
model = ResNet50(include_top=True, weights='imagenet')
x = model.layers[-2].output
x = Dense(train_dataset.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
lr = 0.1
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

# train

In [ ]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190722_0030_TESTNB'

cb_list = list()
cb_list.append(SaveModel(exp_folder, 100))

map_cb = MAPCallback(X_test, Y_test, exp_folder)
cb_list.append(map_cb)

In [ ]:
batch_size=32
nb_epochs=20

steps_per_epoch_train = int(len(train_dataset.id_to_label) / batch_size) + 1
model.fit_generator(train_dataset.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)